In [20]:
import pandas as pd
import gzip

#f = gzip.open("/data/tv/2016/2016-01/2016-01-18/2016-01-18_0100_US_MSNBC_US_Presidential_Politics.v4.vrt.gz", "r")
f = gzip.open("/data/tv/2016/2016-01/2016-01-27/2016-01-27_0735_US_KABC_Jimmy_Kimmel_Live.txt.split.json.gz.vrt.gz", "rt")


# read file and delete lines starting with "<"
lines = f.readlines()
lines = [line for line in lines if not line.startswith('<')]
f.close()
# create a list from lines using "\t" as separator
lines = [line.split("\t") for line in lines]
lines

df = pd.DataFrame(lines)


filtrado = df.iloc[:, [0, 11, 12 ]]

filtrado.head(40)

,0,11,12
0,The,0,0
1,ELKMONT,0,0
2,",",0,0
3,Alabama,0,0
4,half,0,0
5,marathon,1,15
6,.,1,15
7,From,18,19
8,Hollywood,18,19
9,",",18,19


In [ ]:
speech2 = pd.read_csv("2016-01-18_1500_US_CNN_Newsroom.v4.vrt.gz", sep = "\t", header = 6, skiprows = 6, decimal = ".", encoding = "UTF-8")
speech2.head(20)

In [ ]:
# delete rows with first column start with "<" 
#df[df[1].str.contains("this string")==False]
df.columns





#filtrado = speech.iloc[:, [0, 51, 52 ]]
# create new column with column 1 + column 2 * 0.001
# rename columns to "string", "second" and "milisecond"
#filtrado.columns = ["string", "second", "milisecond"]
#filtrado["cutTime"] = filtrado["second"] + filtrado["milisecond"] * 0.001
#filtrado.head(20)


In [ ]:
# print rows with string isin ["whole", "debate"]
print(filtrado[filtrado["string"].isin(["whole"])])

In [ ]:
# get rows with conditions
cond = filtrado["string"].eq("whole") & filtrado["string"].shift(-1).eq("debate")

In [ ]:
cortes = filtrado[cond]
cortes

In [ ]:
# Create new column in cortes called "cutTime" with the value of second + milisecond * 0.001. 
#cortes["cutTime"] = cortes["second"] + (cortes["milisecond"] * 0.001)
cortes["milisecond"] = cortes["milisecond"] * 0.001
#cortes["cutTime"] = cortes.apply(lambda x: x.second + (x.milisecond * 0.001), axis = 1)
cortes["cutTime"] = cortes["second"] + cortes["milisecond"]
cortes


In [ ]:
a = list(cortes["cutTime"])

In [ ]:
print(a)


In [21]:
import gzip
import pandas as pd

# get cut times of a video containing n word searching text and return list with exact second 
# where the first word of text is found.
# Imagine this gentle output:
# [1st match "whole debate"]
#  string  Time
# whole   2125.059
# debate  2125.088
# was     2126.030
#
## [2nd match "whole debate"]
# string  Time
# whole   2133.004
# debate  2133.032
# ,       2133.077
# 
# [3]
# string  Time
# whole   2150.060
# debate  2150.089
# rises   2151.030

# example: get_cut_times("2016-01-18_1500_US_CNN_Newsroom.v4.vrt.gz", "whole") - return: [398.039, 1800.052, 2125.059, 2133.004, 2150.06, 2483.044, 2675.048]
# example: get_cut_times("2016-01-18_1500_US_CNN_Newsroom.v4.vrt.gz", "whole debate") - return: [2125.059, 2133.004, 2150.06]
# example: get_cut_times("2016-01-18_1500_US_CNN_Newsroom.v4.vrt.gz", "whole debate rises") - return: [2150.06]
def get_cut_times(video, search_text):
    cutting_times = {}
  
    # read file and delete lines starting with "<"
    f = gzip.open(video,"rt")
    lines = f.readlines()
    lines = [line for line in lines if not line.startswith('<')]
    f.close()
    # create a list from lines using "\t" as separator
    lines = [line.split("\t") for line in lines]
    gentle = pd.DataFrame(lines)
    # Selection on important columns
    selection1 = gentle.iloc[:, [0, 51, 52 ]]
    # rename columns to "string", "second" and "milisecond"
    selection1.columns = ["string", "second", "milisecond"]
    selection1["string"] = selection1["string"].apply(lambda x: x.lower())
    search_text = search_text.lower()
    words = search_text.split(" ")
    # get rows with conditions. consecutive words must be in consecutive rows
    for i in range(len(words)):
      if i == 0:
        cond = selection1["string"].eq(words[i])
      else:
        cond = cond & selection1["string"].shift(-1*i).eq(words[i])
    cutting_times = selection1[cond]
    # Create new column in cutting_times called "cutTime" with the value of second + milisecond * 0.001.
    print(cutting_times["string"] + " --> " + cutting_times["second"] + ":" + cutting_times["milisecond"] )
    try:
      cutting_times["second"]     = cutting_times["second"].astype(float)
    except:
      print("Error: second column is not a float")
      cutting_times = {}
      return list(cutting_times)
    try:
      cutting_times["milisecond"] = cutting_times["milisecond"].astype(float) * 0.001
    except:
      print("Error: milisecond column is not a float")
      cutting_times = {}
      return list(cutting_times)

    cutting_times["cutTime"] = cutting_times["second"] + cutting_times["milisecond"]
    # print next row
    
    return list(cutting_times["cutTime"])


# print(get_cut_times("tmp/2016-09-01_2200_US_KNBC_The_Ellen_DeGeneres_Show.v4.vrt.gz", "new game"))

#print(get_cut_times("/data/tv/2016/2016-01/2016-01-27/2016-01-27_0000_US_KNBC_The_Ellen_DeGeneres_Show.v4.vrt.gz", "how are you"))
#print(get_cut_times("/data/tv/2016/2016-01/2016-01-27/2016-01-27_0000_US_KNBC_The_Ellen_DeGeneres_Show.v4.vrt.gz", "how are you"))
print(get_cut_times("/data/tv/2016/2016-01/2016-01-27/2016-01-27_0735_US_KABC_Jimmy_Kimmel_Live.txt.split.json.gz.vrt.gz", "how are you"))
print(get_cut_times("/data/tv/2016/2016-01/2016-01-27/2016-01-27_0735_US_KABC_Jimmy_Kimmel_Live.txt.split.json.gz.vrt.gz", "how are you"))

1106    how --> 576:580
dtype: object
[576.58]
4165    how --> 1751:1752
4200    how --> 1762:1763
6920    how --> 2917:2918
dtype: object
[1752.752, 1763.763, 2919.918]


/tmp/ipykernel_222461/958051083.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selection1["string"] = selection1["string"].apply(lambda x: x.lower())
/tmp/ipykernel_222461/958051083.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cutting_times["second"]     = cutting_times["second"].astype(float)
/tmp/ipykernel_222461/958051083.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [9]:
c = {}

c["a"] = []

list(c)

['a']

In [ ]:
def get_all_vrts(videos_folder):
    vrts = []
    for root, dirs, files in os.walk(videos_folder):
        for file in files:
            if file.endswith(".vrt.gz"):
                vrts.append(os.path.join(root, file))
    return vrts


# get all vrts in a folder

vrts = get_all_vrts("/data/tv/2016/2016-09")
vrts